# __这个文件不需要运行__

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime
from threading import Thread

In [2]:
def parse_page():
    global Data
    global parse_sleep
    
    while True:
        sleep(parse_sleep)
        
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="j_market_card"]')))

        try:
            next_page = driver.find_elements('xpath', '/html/body/div[5]/div[1]/div[4]/div[2]/ul/li')[-1]
            source = driver.execute_script("return document.documentElement.outerHTML")
        except:
            sleep(5)
            source = driver.execute_script("return document.documentElement.outerHTML")
            try:
                next_page = driver.find_elements('xpath', '/html/body/div[5]/div[1]/div[4]/div[2]/ul/li')[-1]
            except:
                Data += get_data(source)
                break
        Data += get_data(source)

        if next_page.get_attribute('class')=='':
            next_page.click()
        else:
            break

In [3]:
def get_data(source):
    global time
    
    all_data = []
    # 解析HTML代码
    soup = BeautifulSoup(source, 'html.parser')

    # 获取符合条件的所有<li>元素
    li_elements = soup.select('ul.card_csgo li')

    # 获取每个<li>元素中的需要的内容
    for li in li_elements:
        try:
            # 获取游戏名称
            full_name = li.select_one('a')['title'].strip()
            name_without_wear = re.sub(r'\([^)]*\)', '', full_name)
            pure_name = name_without_wear
            if "★ StatTrak™" in name_without_wear:
                pure_name = name_without_wear.replace("★ StatTrak™", "")
            if "Souvenir" in name_without_wear:
                pure_name = name_without_wear.replace("Souvenir", "")
            if "StatTrak™" in name_without_wear:
                pure_name = name_without_wear.replace("StatTrak™", "")
            if "★" in name_without_wear:
                pure_name = name_without_wear.replace("★", "")
            
            # 获取category
            category_list = ['★', 'Souvenir', 'StatTrak™', '★ StatTrak™']
            cate = 'Normal'
            for category in category_list:
                if category in full_name:
                    cate = category
                    
            # 获取type
            try:
                type_ = pure_name[:pure_name.index('|')-1].strip()
            except:
                type_ = 'None'
                
            # skin_name
            try:
                skin_name = name_without_wear[name_without_wear.index('|')+1 :].strip()
            except:
                skin_name = name_without_wear
        
            # 获取价格
            price = li.select_one('strong.f_Strong').text.strip()[2:]

            # 获取数量
            num = li.select_one('span.l_Right').text.strip()

            # 获取衣服类别
            cloth_info = re.search(r'\((.*?)\)', full_name).group(1)

            # 将数据保存到字典中
            data = {
                'Time' : time,
                'Full_name': full_name,
                'Skin_name': skin_name,
                'Price': price,
                'Category':cate,
                'type':type_,
                'On Sale': num,
                'Exterior': cloth_info
            }
            all_data.append(data)
        except:
            continue

    return all_data

In [4]:
now = datetime.now()
time = f'{now.year}-{now.month}-{now.day} {now.hour}:{now.minute}'

In [ ]:
Data = []

parse_sleep = 2.5

url = 'https://buff.163.com/market/csgo#tab=selling&page_num=1'

# 初始化
options = webdriver.ChromeOptions()
options.add_argument(r"--user-data-dir=/home/nolan/.config/google-chrome/Default")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 5)

print(f'Time is \033[1;32m{time}\033[0m'.center(50, '='))
print('Start collecting \033[1;32mbuff.163.com\033[0m'.center(50, ' '))

driver.get(url)

# 打开筛选
wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[1]/div[3]/div[1]/div[3]'))).click()

sort_button = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="quantity_li"]/h6'))).click()

#
c_list = driver.find_elements(By.XPATH, '//*[@id="j_h1z1-selType"]/div')
    
for i in c_list[:5]:
    
    for j in driver.find_elements(By.XPATH, f'//*[@id="j_h1z1-selType"]/div[{c_list.index(i) +1}]/ul/li'):
        try:
            i.click()
            j.click()
            parse_page()
            sleep(3)
            print(f'Current Data Length : \033[1;32m{len(Data)}\033[0m'.center(50, ' '), end='\r')
        except:
            continue

driver.quit()
# parse_page()

# print(f'\033[1;32m{name}\033[0m Successfully Collected !')

# driver.quit()

# Data = pd.DataFrame(Data)
# Data['Kind'] = name
# Data.to_csv(f'./price_data/{time}({name}).csv', index=False)
# print(f'\033[1;32m{name}\033[0m Saved !')

========Time is 2023-5-28 15:21========
     Start collecting buff.163.com     


就是上面这个时候(已经收集了8884条数据)我被禁止访问市场数据的。大概是第四次完整测试selenium,我发现虽然绝大多数数据都能收集到，但是有一千条左右的不一定能收集到，我在调参数尝试能不能解决时被封了。

In [ ]:
pd.DataFrame(Data).to_csv(f'./price_data/{time}.csv', index=False)